In [27]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113


In [29]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx


%cd /content/drive/MyDrive/ADNI2/Controls
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      centrality = nx.betweenness_centrality(G)
      x = []
      for i in range(len(centrality)):
        x.append([centrality[i]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([1])
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

%cd /content/drive/MyDrive/ADNI2/Patients
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      centrality = nx.betweenness_centrality(G)
      x = []
      for i in range(len(centrality)):
        x.append([centrality[i]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([0])
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Controls
/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Patients


In [30]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [31]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch 
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [32]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 1.1225
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6988
Epoch: 003, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6926
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7207
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7033
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7155
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7432
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6935
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7134
Epoch: 010, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6929
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7127
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6863
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7039
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7052
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7100
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6928
Epoch: 0

In [33]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [34]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6925
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6943
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6928
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6812
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7041
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7029
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6812
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7107
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6876
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7029
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6883
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6886
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6966
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6848
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6886
Epoch: 0

In [35]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [36]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.5525
Epoch: 002, Train Acc: 0.4800, Test Acc: 0.5588, Loss: 0.3271
Epoch: 003, Train Acc: 0.4800, Test Acc: 0.5441, Loss: 0.3665
Epoch: 004, Train Acc: 0.5667, Test Acc: 0.6029, Loss: 0.4369
Epoch: 005, Train Acc: 0.7867, Test Acc: 0.8382, Loss: 0.2365
Epoch: 006, Train Acc: 0.5400, Test Acc: 0.5882, Loss: 0.2636
Epoch: 007, Train Acc: 0.5200, Test Acc: 0.5735, Loss: 0.3724
Epoch: 008, Train Acc: 0.5533, Test Acc: 0.5882, Loss: 0.2803
Epoch: 009, Train Acc: 0.5667, Test Acc: 0.5882, Loss: 0.2564
Epoch: 010, Train Acc: 0.5733, Test Acc: 0.6029, Loss: 0.3417
Epoch: 011, Train Acc: 0.5733, Test Acc: 0.6471, Loss: 0.2907
Epoch: 012, Train Acc: 0.6467, Test Acc: 0.7353, Loss: 0.4885
Epoch: 013, Train Acc: 0.8067, Test Acc: 0.8382, Loss: 0.4301
Epoch: 014, Train Acc: 0.7333, Test Acc: 0.7647, Loss: 0.5035
Epoch: 015, Train Acc: 0.6800, Test Acc: 0.7647, Loss: 0.1978
Epoch: 016, Train Acc: 0.8467, Test Acc: 0.8676, Loss: 0.3194
Epoch: 0

In [37]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [38]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7669
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7030
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6897
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6880
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6844
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6908
Epoch: 007, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.6785
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6242
Epoch: 009, Train Acc: 0.6133, Test Acc: 0.5294, Loss: 0.5792
Epoch: 010, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.5146
Epoch: 011, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.3951
Epoch: 012, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.4427
Epoch: 013, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.3568
Epoch: 014, Train Acc: 0.8133, Test Acc: 0.8382, Loss: 0.3100
Epoch: 015, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.1962
Epoch: 016, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.3130
Epoch: 0

In [39]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [40]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6871
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7054
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6925
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6923
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6995
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6966
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7114
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6879
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6879
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7064
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6842
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6966
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6885
Epoch: 0

In [41]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [42]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7325
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6931
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6892
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6843
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6784
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6746
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6630
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6778
Epoch: 009, Train Acc: 0.8067, Test Acc: 0.8235, Loss: 0.5967
Epoch: 010, Train Acc: 0.8067, Test Acc: 0.8235, Loss: 0.4062
Epoch: 011, Train Acc: 0.7867, Test Acc: 0.8088, Loss: 0.4273
Epoch: 012, Train Acc: 0.6133, Test Acc: 0.5147, Loss: 0.4370
Epoch: 013, Train Acc: 0.8600, Test Acc: 0.8235, Loss: 0.3135
Epoch: 014, Train Acc: 0.8600, Test Acc: 0.8235, Loss: 0.2832
Epoch: 015, Train Acc: 0.8800, Test Acc: 0.8235, Loss: 0.3754
Epoch: 016, Train Acc: 0.8733, Test Acc: 0.8235, Loss: 0.5478
Epoch: 0

In [43]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [44]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6734
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6943
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6752
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6954
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6785
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6849
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6755
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6821
Epoch: 009, Train Acc: 0.9133, Test Acc: 0.8676, Loss: 0.6648
Epoch: 010, Train Acc: 0.6133, Test Acc: 0.5441, Loss: 0.6150
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.4864
Epoch: 012, Train Acc: 0.6933, Test Acc: 0.7500, Loss: 0.5909
Epoch: 013, Train Acc: 0.8333, Test Acc: 0.8235, Loss: 0.3880
Epoch: 014, Train Acc: 0.9467, Test Acc: 0.9265, Loss: 0.3699
Epoch: 015, Train Acc: 0.9467, Test Acc: 0.8824, Loss: 0.1686
Epoch: 016, Train Acc: 0.9333, Test Acc: 0.8676, Loss: 0.2358
Epoch: 0

In [45]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [46]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6901
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6921
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6884
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7128
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6767
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6946
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6759
Epoch: 009, Train Acc: 0.6133, Test Acc: 0.5294, Loss: 0.5745
Epoch: 010, Train Acc: 0.6133, Test Acc: 0.5294, Loss: 0.5017
Epoch: 011, Train Acc: 0.8000, Test Acc: 0.8235, Loss: 0.4938
Epoch: 012, Train Acc: 0.8000, Test Acc: 0.8235, Loss: 0.4774
Epoch: 013, Train Acc: 0.8000, Test Acc: 0.8235, Loss: 0.1874
Epoch: 014, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.4584
Epoch: 015, Train Acc: 0.8000, Test Acc: 0.8235, Loss: 0.3991
Epoch: 016, Train Acc: 0.8600, Test Acc: 0.8382, Loss: 0.3948
Epoch: 0

In [47]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [48]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6961
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7031
Epoch: 003, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7203
Epoch: 004, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7025
Epoch: 005, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6956
Epoch: 006, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6831
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6864
Epoch: 008, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6924
Epoch: 009, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6787
Epoch: 010, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6871
Epoch: 011, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6924
Epoch: 012, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6973
Epoch: 013, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6901
Epoch: 014, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6925
Epoch: 015, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6883
Epoch: 016, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6845
Epoch: 0

In [49]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [50]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7526
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6864
Epoch: 003, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7066
Epoch: 004, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7057
Epoch: 005, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7171
Epoch: 006, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6813
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7087
Epoch: 008, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7027
Epoch: 009, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6873
Epoch: 010, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6918
Epoch: 011, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6999
Epoch: 012, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7141
Epoch: 013, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6980
Epoch: 014, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6911
Epoch: 015, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6888
Epoch: 016, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6930
Epoch: 0

In [51]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [52]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6982
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6856
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6877
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6876
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6836
Epoch: 006, Train Acc: 0.9600, Test Acc: 0.9412, Loss: 0.7848
Epoch: 007, Train Acc: 0.6933, Test Acc: 0.7353, Loss: 0.5844
Epoch: 008, Train Acc: 0.7600, Test Acc: 0.7941, Loss: 0.4079
Epoch: 009, Train Acc: 0.9600, Test Acc: 0.9265, Loss: 0.4611
Epoch: 010, Train Acc: 0.5400, Test Acc: 0.6176, Loss: 1.3997
Epoch: 011, Train Acc: 0.5333, Test Acc: 0.4559, Loss: 0.3280
Epoch: 012, Train Acc: 0.7867, Test Acc: 0.7941, Loss: 0.5350
Epoch: 013, Train Acc: 0.8333, Test Acc: 0.8382, Loss: 0.3231
Epoch: 014, Train Acc: 0.9533, Test Acc: 0.8971, Loss: 0.3315
Epoch: 015, Train Acc: 0.8333, Test Acc: 0.8382, Loss: 0.5417
Epoch: 016, Train Acc: 0.9533, Test Acc: 0.8824, Loss: 0.3251
Epoch: 0